In [1]:
# module set
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from collections import OrderedDict
from itertools import repeat

In [2]:
import os

In [3]:
driver = webdriver.Chrome("chromedriver")
driver.implicitly_wait(3)
positions = ['st', 'mid', 'def', 'gl']
player_url = 'http://fifaonline4.inven.co.kr/dataninfo/player/?code='

### **Players' DB**

In [4]:
def position_db(pos):
    # 선수 DB 페이지로 이동

    driver.get('http://fifaonline4.inven.co.kr/dataninfo/player/')
    driver.implicitly_wait(5)
    # 포지션 체크// checkbox의 xpath가 아닌 span 태그의 xpath를 긁어와야 함.
    if pos == 'st':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[1]/div[2]/label[1]/span')
        checkbox.click()
    if pos == 'mid':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[2]/div[2]/label[1]/span')
        checkbox.click()
    if pos == 'def':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[3]/div[2]/label[1]/span')
        checkbox.click()
    if pos =='gl':
        checkbox = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[3]/div[2]/div[4]/div[2]/label[1]/span')
        checkbox.click()
    search = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[2]/li[17]/button[1]/img')
    search.click()

In [5]:
def players_info(html):
    soup = bs(html, 'html.parser')

    search_url_list = []
    for link in soup.find_all('a'):
        if 'href' in link.attrs:
            li = link.attrs['href'].split('\n')
            if li[0].startswith(player_url):
                search_url_list.append(li[0])
            else:
                continue

    search_url_list = list(OrderedDict(zip(search_url_list, repeat(None))))
    print("num of url : ", len(search_url_list))
    print(search_url_list)
    return search_url_list

In [6]:
def StatDB(url_list):
    total_df=pd.DataFrame()
    
    for search_url in url_list: # 아까 검색해야할 url 리스트에서 하나씩 가져온다
        one_player_page = requests.get(search_url)
        one_soup=bs(one_player_page.content,'lxml')
        # one_player_db 는 각 선수마다 데이터베이스. 루프 한번 돌 때마다 각 선수의 DB를 일단 긁어옴
        one_player_db=one_soup.find_all("div", {"class":"fifa4 db_tooltip"})
    
        for db in one_player_db:
            detail_dict={}  
            for i,p in enumerate(db.find_all("p")):
                if i<10:
                    if i==0:
                        detail_dict['name']=p.text
                    elif i==7:
                        detail_dict['foot_left']=p.text
                    elif i==8:
                        detail_dict['foot_right_act']=p.text
                    elif i==9:
                        detail_dict['specialty']=p.text
                else:
                    if i%2==0: #홀수면 스탯 이름, 짝수면 스탯 값
                        key=p.text
                    elif i%2!=0:
                        detail_dict[key]=p.text
                    
        one_df=pd.DataFrame.from_dict(detail_dict, orient='index')
        total_df=pd.concat([total_df,one_df], axis=1)
    
    total_df=total_df.T
    return total_df

#### Striker DB

In [7]:
position_db('st')

In [8]:
urls = players_info(driver.page_source)

num of url :  500
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=101190045', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001114', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101242519', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101247553', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101031432', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001075', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101004000', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101004231', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101045661', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101007512', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101214267', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101247703', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101248146', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001201', 'http://fifaonline4.inven.co.kr/dataninfo/p

In [9]:
ST_db = StatDB(urls)
ST_db

,name,foot_left,foot_right_act,specialty,스피드,슛,패스,드리블,수비,피지컬,...,몸싸움,스태미너,적극성,점프,침착성,GK 다이빙,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정
0,요한 크루이프,5,5,"특성화려한 개인기, 플레이 메이커, 아웃사이드 슈팅/크로스, 칩슛 선호, 테크니컬 ...",109,109,109,117,70,100,...,96,106,106,91,113,23,27,22,29,25
0,로베르토 바조,5,5,"특성예리한 감아차기, 플레이 메이커, 테크니컬 드리블러",106,104,110,116,59,86,...,86,101,68,85,108,22,32,25,27,24
0,에우제비우 다 실바 페헤이라,5,5,"특성화려한 개인기, 중거리 슛 선호, 스피드 드리블러, 플레이 메이커",116,114,103,117,65,101,...,101,108,95,109,111,28,22,26,30,31
0,마누엘 프란시스코 두스 산투스,4,5,"특성화려한 개인기, 스피드 드리블러, 플레이 메이커, 테크니컬 드리블러",117,103,107,121,63,93,...,86,105,97,99,116,19,21,20,16,18
0,디디에 드로그바,4,5,"특성다이버, 강철몸, 화려한 개인기, 중거리 슛 선호, 파워 헤더, 승부욕",107,113,93,106,64,114,...,118,105,117,111,111,23,24,22,21,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,스테판 엘샤라위,4,5,"특성예리한 감아차기, 화려한 개인기, 스피드 드리블러",96,89,86,91,53,70,...,61,93,64,79,78,10,13,15,13,11
0,스타니슬라프 일리우첸코,4,5,특성,88,90,72,82,51,91,...,95,93,82,87,95,13,19,14,12,17
0,앙헬 디마리아,5,2,"특성화려한 개인기, 중거리 슛 선호, 아웃사이드 슈팅/크로스, 칩슛 선호, 테크니컬...",90,85,90,93,57,75,...,69,85,80,80,90,24,22,25,25,26
0,마리오 발로텔리,4,5,"특성개인 플레이 선호, 화려한 개인기, 중거리 슛 선호, 아웃사이드 슈팅/크로스, ...",86,94,84,94,27,84,...,92,74,79,83,91,5,5,5,5,6


#### Midfielder DB

In [10]:
driver.back()

In [11]:
position_db('mid')

In [12]:
urls = players_info(driver.page_source)

num of url :  500
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001397', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001114', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001075', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101004000', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101004231', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001668', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101005471', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101013743', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001620', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101190046', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001845', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101222680', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220208722', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220228702', 'http://fifaonline4.inven.co.kr/dataninfo/p

In [13]:
MF_db = StatDB(urls)
MF_db

,name,foot_left,foot_right_act,specialty,스피드,슛,패스,드리블,수비,피지컬,...,몸싸움,스태미너,적극성,점프,침착성,GK 다이빙,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정
0,지네딘 지단,5,5,"특성예리한 감아차기, 화려한 개인기, 긴 패스 선호, 플레이 메이커, 패스 마스터,...",109,108,110,117,89,104,...,104,106,108,97,119,18,21,19,21,28
0,로베르토 바조,5,5,"특성예리한 감아차기, 플레이 메이커, 테크니컬 드리블러",106,104,110,116,59,86,...,86,101,68,85,108,22,32,25,27,24
0,알레산드로 델피에로,5,5,"특성예리한 감아차기, 중거리 슛 선호, 플레이 메이커, 테크니컬 드리블러",109,109,107,112,65,96,...,93,106,95,91,114,20,16,20,21,29
0,데니스 베르캄프,4,5,"특성예리한 감아차기, 화려한 개인기, 칩슛 선호",106,106,102,112,64,101,...,104,106,89,96,109,29,23,27,22,17
0,히바우두,5,2,"특성다이버, 주발 선호, 개인 플레이 선호, 예리한 감아차기, 화려한 개인기, 중거...",106,109,108,111,62,98,...,100,103,88,94,108,29,27,32,29,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,루이스 구스타부 지아스,5,3,"특성슬라이딩 태클 선호, 긴 패스 선호",72,64,79,80,89,93,...,92,95,97,82,91,12,8,13,15,14
0,로이 킨,3,5,"특성슬라이딩 태클 선호, 중거리 슛 선호",73,83,83,79,91,91,...,92,88,99,82,88,11,17,15,17,17
0,무사 뎀벨레,5,2,"특성화려한 개인기, 중거리 슛 선호, 테크니컬 드리블러",87,82,86,101,85,92,...,98,87,85,87,102,19,17,14,17,19
0,무사 시소코,3,5,"특성강철몸, 주발 선호, 스피드 드리블러",91,77,86,94,89,99,...,101,100,97,90,80,17,11,14,11,13


#### Defender DB

In [14]:
driver.back()

In [15]:
position_db('def')

In [16]:
urls = players_info(driver.page_source)

num of url :  500
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001040', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101001183', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101005099', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101046747', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220203376', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220155862', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220216267', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220231281', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220235243', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224155862', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=225138161', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=225140053', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224001183', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224166906', 'http://fifaonline4.inven.co.kr/dataninfo/p

In [17]:
DF_db = StatDB(urls)
DF_db

,name,foot_left,foot_right_act,specialty,스피드,슛,패스,드리블,수비,피지컬,...,몸싸움,스태미너,적극성,점프,침착성,GK 다이빙,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정
0,호베르투 카를루스 다 시우바 주니오르,5,2,"특성장거리 스로인, 주발 선호, 예리한 감아차기, 중거리 슛 선호, 승부욕",116,106,103,107,107,106,...,103,115,106,105,102,16,18,26,19,22
0,파비오 칸나바로,3,5,"특성강철몸, 슬라이딩 태클 선호, 화려한 걷어내기",102,67,83,78,112,105,...,106,98,109,114,100,28,23,31,30,30
0,잔루카 잠브로타,5,5,"특성슬라이딩 태클 선호, 얼리 크로스 선호, 스피드 드리블러",109,91,97,110,107,100,...,93,111,106,93,98,27,29,28,16,24
0,알렉시 랄라스,3,5,"특성슬라이딩 태클 선호, 파워 헤더",94,74,88,82,103,104,...,111,97,99,99,98,25,18,28,28,24
0,버질 반데이크,3,5,"특성강철몸, 긴 패스 선호, 파워 헤더",102,86,91,88,104,101,...,106,92,104,103,101,16,13,16,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,제이손 무리요,3,5,"특성슬라이딩 태클 선호, 파워 헤더",86,42,65,66,88,89,...,91,82,93,93,83,15,14,9,15,12
0,존 스톤스,3,5,특성강철몸,83,56,76,78,87,86,...,88,88,84,81,86,10,18,14,11,13
0,제라르 피케 베르나베우,3,5,특성,61,66,75,68,90,82,...,89,74,79,78,89,20,21,24,19,25
0,하창래,3,5,특성,98,56,62,67,87,90,...,91,81,98,100,68,12,9,13,11,17


#### GK DB

In [24]:
driver.back()

In [29]:
position_db('gl')

In [30]:
urls = players_info(driver.page_source)

num of url :  500
['http://fifaonline4.inven.co.kr/dataninfo/player/?code=101000805', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=101000001', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=220212831', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224001179', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=225019580', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224238380', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224167495', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=222200389', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=211193080', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=224167948', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=225136594', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=221193041', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=217212831', 'http://fifaonline4.inven.co.kr/dataninfo/player/?code=222192448', 'http://fifaonline4.inven.co.kr/dataninfo/p

In [31]:
GK_db = StatDB(urls)
GK_db

,name,foot_left,foot_right_act,specialty,다이빙,핸들링,킥,반응속도,스피드,위치선정,...,몸싸움,스태미너,적극성,점프,침착성,GK 다이빙,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정
0,옌스 레만,3,5,"특성GK 멀리 던지기, 스위퍼 키퍼, GK 적극적 크로스 수비",105,109,104,104,78,108,...,98,63,105,91,91,105,109,104,104,108
0,데이비드 시먼,3,5,"특성GK 능숙한 펀칭, GK 멀리 던지기",101,108,109,104,63,106,...,85,73,54,80,89,101,108,109,104,106
0,알리송 함시스 베케르,3,5,"특성GK 멀리 던지기, 스위퍼 키퍼",101,101,97,104,62,104,...,88,35,40,83,77,101,101,97,104,104
0,잔루이지 부폰,2,5,"특성GK 능숙한 펀칭, GK 침착한 1:1 수비, GK 소극적 크로스 수비",101,97,92,100,53,108,...,72,37,41,91,87,101,97,92,100,108
0,이운재,2,5,"특성GK 능숙한 펀칭, GK 침착한 1:1 수비",96,104,94,102,45,105,...,90,50,72,82,75,96,104,94,102,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,에두아르 멘디,2,5,특성GK 적극적 크로스 수비,81,77,75,82,43,80,...,72,44,32,47,62,81,77,75,82,80
0,사미르 한다노비치,2,5,"특성GK 멀리 던지기, GK 침착한 1:1 수비",81,78,64,82,49,82,...,66,39,25,72,64,81,78,64,82,82
0,슈벤 울라이히,2,5,특성,80,74,71,86,45,81,...,55,35,32,68,50,80,74,71,86,81
0,조르디 마시프 로페스,3,5,특성GK 능숙한 펀칭,83,78,78,82,64,77,...,59,47,29,83,59,83,78,78,82,77


In [35]:
ST_db = ST_db.set_index('name')
ST_db.to_csv('ST_db.csv', encoding='ms949')

In [36]:
MF_db = MF_db.set_index('name')
MF_db.to_csv('MF_db.csv', encoding='ms949')

In [37]:
DF_db = DF_db.set_index('name')
DF_db.to_csv('DF_db.csv', encoding='ms949')

In [40]:
GK_db = GK_db.set_index('name')
GK_db.to_csv('GK_db.csv', encoding='ms949')

In [41]:
FinalDB = pd.concat([ST_db, MF_db, DF_db, GK_db])
FinalDB

,foot_left,foot_right_act,specialty,스피드,슛,패스,드리블,수비,피지컬,속력,...,GK 다이빙,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정,다이빙,핸들링,킥,반응속도,위치선정
name,,,,,,,,,,,,,,,,,,,,,
요한 크루이프,5,5,"특성화려한 개인기, 플레이 메이커, 아웃사이드 슈팅/크로스, 칩슛 선호, 테크니컬 ...",109,109,109,117,70,100,107,...,23,27,22,29,25,NaN,NaN,NaN,NaN,NaN
로베르토 바조,5,5,"특성예리한 감아차기, 플레이 메이커, 테크니컬 드리블러",106,104,110,116,59,86,105,...,22,32,25,27,24,NaN,NaN,NaN,NaN,NaN
에우제비우 다 실바 페헤이라,5,5,"특성화려한 개인기, 중거리 슛 선호, 스피드 드리블러, 플레이 메이커",116,114,103,117,65,101,118,...,28,22,26,30,31,NaN,NaN,NaN,NaN,NaN
마누엘 프란시스코 두스 산투스,4,5,"특성화려한 개인기, 스피드 드리블러, 플레이 메이커, 테크니컬 드리블러",117,103,107,121,63,93,119,...,19,21,20,16,18,NaN,NaN,NaN,NaN,NaN
디디에 드로그바,4,5,"특성다이버, 강철몸, 화려한 개인기, 중거리 슛 선호, 파워 헤더, 승부욕",107,113,93,106,64,114,106,...,23,24,22,21,19,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
에두아르 멘디,2,5,특성GK 적극적 크로스 수비,43,NaN,NaN,24,NaN,NaN,42,...,81,77,75,82,80,81,77,75,82,80
사미르 한다노비치,2,5,"특성GK 멀리 던지기, GK 침착한 1:1 수비",49,NaN,NaN,13,NaN,NaN,51,...,81,78,64,82,82,81,78,64,82,82
슈벤 울라이히,2,5,특성,45,NaN,NaN,21,NaN,NaN,45,...,80,74,71,86,81,80,74,71,86,81


In [44]:
FinalDB.duplicated(['name'])

KeyError: Index(['name'], dtype='object')

In [ ]:
# st = ['st', 'cf', 'rw', 'lw']
# mid = ['cm', 'cam', 'cdm', 'lm', 'rm']
# df = ['cb','rb','lb', 'rwb', 'lwb']

# def get_player_stats(url):
    